In [1]:
# !pip install torch

In [2]:
# !pip3 install pdfminer3
# !pip3 install contractions
# !pip3 install spacy
# !pip3 install unicodedata
# !pip3 install nltk
# !pip3 install numpy
# !pip3 install string
# !pip3 install pandas
# !pip3 install gensim
# !pip3 install sklearn
# !pip3 install matplotlib
# !pip3 install plotly
# !pip3 install torch
# !pip3 install torchvision
# !pip install --upgrade gensim


In [3]:
# !pip3 install transformers
# !pip3 install sentence_transformers

In [4]:
# !pip3 install 

In [5]:
import itertools
from nlp_functions import text_loader, classifier, remove_strange_characters, remove_n, remove_colons, remove_stripes, remove_redundant_whitespaces, remove_digits
import torchvision
import torch
import plotly.express as px
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import matplotlib
from sklearn.manifold import TSNE
import gensim.downloader as api
from spacy.lang.en import English
import pandas as pd
from gensim.models import Word2Vec
import os
import shutil
import io
from pdfminer3.converter import TextConverter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfpage import PDFPage
from pdfminer3.layout import LAParams, LTTextBox
import contractions
import string
import numpy as np
import spacy
import unicodedata
import re

import collections

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:


def pdf_loader(company_name):
    company_name = company_name[:-4]
    source = 'D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Sustainability Reports\To Do'
    path = rf"{source}\{company_name}.pdf"

    text = text_loader(path)

    return text, company_name, path, source


def preprocessing_df(text):
    text = re.sub(r'\d+', '', text)

    sentences = []
    for s in sent_tokenize(text):
        sentences.append(s)

    df = pd.DataFrame(sentences, columns=['Sentences'])

    df['Alt_Text'] = df['Sentences'].apply(
        lambda x: remove_strange_characters(x))
    df['Alt_Text'] = df['Alt_Text'].apply(lambda x: remove_n(x))
    df['Alt_Text'] = df['Alt_Text'].apply(lambda x: remove_stripes(x))
    df['Alt_Text'] = df['Alt_Text'].apply(
        lambda x: remove_redundant_whitespaces(x))

    return df


def classifier_pipeline(dataframe):
    token_ls = dataframe["Alt_Text"].tolist()
    threshold = 0.5
    classification_ls = []
    short_ls = token_ls[:]
    print(len(short_ls))
    for i in short_ls:  # removing undedected empty vectors
        if len(i) < 1:
            short_ls.remove(i)
    print(len(short_ls))

    for i in short_ls:
        # token_str = ' '.join(i)
        x = classifier(i)
        if x.get("scores")[0] > threshold:
            classification_ls.append(x.get("labels")[0])
        else:
            classification_ls.append("other")
        classification_ls

    return classification_ls, short_ls


def rebuilding(classification_ls, short_ls):
    sustainability_text = []
    human_rights_text = []
    fraud_text = []
    social_issue_text = []
    employee_affairs_text = []
    other_text = []

    for (text, label) in zip(classification_ls, short_ls):
        if label == "sustainability":
            sustainability_text.append(text)
        elif label == "human rights":
            human_rights_text.append(text)
        elif label == "fraud":
            fraud_text.append(text)
        elif label == "social issues":
            social_issue_text.append(text)
        elif label == "labour law":
            employee_affairs_text.append(text)
        elif label == "other":
            other_text.append(text)
        else:
            continue

    nested_label_list = [sustainability_text, human_rights_text,
                         fraud_text, social_issue_text, employee_affairs_text, other_text]

    return nested_label_list


def list_to_string_join(input_list):
    string = []
    for item in input_list:
        string.append(item)
    joined_string = ' '.join(string)

    return joined_string


def labeling_nested_list(input_list):
    nested_label_list = input_list
    label_string = []
    for label in nested_label_list:
        label_string.append(list_to_string_join(label))

    label_candidate = ['sustainability', 'human rights',
                       'fraud', 'social issues', 'labour law', "others"]
    tuples_prep = list(zip(label_candidate, label_string))
    report_df = pd.DataFrame(tuples_prep, columns=["Label", "Text"])
    return report_df


def similarity_function(text_1, text_2):
    sentences = [text_1, text_2]

    # !pip install sentence_transformers
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(sentences)
    sentence_embeddings.shape
    return cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
    )


def scoring(report_df):
    wiki_text = pd.read_csv(
        r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\wiki_artikel_summary.csv")
    result_cosine_similarity = []
    wiki_text_list = wiki_text["Text"].to_list()
    wiki_labels_list = wiki_text["Label"].to_list()
    report_text_list = report_df["Text"].to_list()

    for w_text, r_text in zip(wiki_text_list, report_text_list):
        pre_result = (similarity_function(w_text, r_text))
        result_cosine_similarity.append(pre_result[0][0])

    data_tuples = list(zip(wiki_labels_list, result_cosine_similarity))
    result_df = pd.DataFrame(data_tuples, columns=['Label', 'Score'])
    return result_df


def preprocessing_text(text):
    # mapping = str.maketrans('', '', string.digits)
    # text = text.translate(mapping)

    text = remove_strange_characters(text)
    text = re.sub(r'\d+', '', text)

    text = remove_n(text)

    text = remove_colons(text)

    text = text.replace(r" .", ".")
    text = text.replace(r"..", ".")
    text = text.replace(r"...", ".")

    text = remove_stripes(text)

    text = remove_redundant_whitespaces(text)
    return text


def topic_modeler(text):

    classifier_pipeline = pipeline(
        "zero-shot-classification", model="facebook/bart-large-mnli")
    input_sequence = text
    label_candidate = ['sustainability', 'human rights',
                       'fraud', 'social issues', 'labour law']
    x = classifier_pipeline(input_sequence, label_candidate)
    result_label = x["labels"]
    result_score = x["scores"]
    tuple_for_df = list(zip(result_label, result_score))
    df_topic_modeling_score = pd.DataFrame(
        tuple_for_df, columns=["Label", "Score"])
    return df_topic_modeling_score


def similarity_function(text_1, text_2):
    sentences = [text_1, text_2]

    # !pip install sentence_transformers
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(sentences)
    sentence_embeddings.shape
    return cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
    )


def results_for_each_topic(text):
    wiki_text = pd.read_csv(
        r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\wiki_artikel_summary.csv")

    result_cosine_similarity = []
    labels = wiki_text["Label"].to_list()
    for label_text in wiki_text["Text"]:
        pre_result = (similarity_function(label_text, text))
        result_cosine_similarity.append(pre_result[0][0])

    data_tuples = list(zip(labels, result_cosine_similarity))
    result_df = pd.DataFrame(data_tuples, columns=['Label', 'Score'])
    return result_df



 





TI-IDF Functions

In [7]:
def preprocessing(text):
    # mapping = str.maketrans('', '', string.digits)
    # text = text.translate(mapping)

    text = remove_strange_characters(text)
    text = re.sub(r'\d+', '', text)


    text = remove_n(text)

    text = remove_colons(text)

    text = text.replace(r" .",".")
    text = text.replace(r"..",".")
    text = text.replace(r"...",".")

    text = remove_stripes(text)

    text = remove_redundant_whitespaces(text)
    return text


nltk.download('omw-1.4')
nltk.download('wordnet')

def lemmatize_words(text):

    lemmatizer = WordNetLemmatizer()
    text = word_tokenize(text)

    lemma_list = []

    for word in text:
        lemma_word = lemmatizer.lemmatize(word)
        lemma_list.append(lemma_word)

    lemma_text = ' '.join(lemma_list)


    return lemma_text



def remove_small_tokens(prep_text):
    prep_text_before = prep_text
    # print(f"Text before cleaning:  {len(prep_text_before)}")
    # print(type(prep_text_before))
    # prep_text_before = [prep_text_before]
    prep_text_before = word_tokenize(prep_text_before)
    # print(prep_text_before)

    for word in prep_text_before:
        if len(word) <= 3:
            prep_text_before.remove(word)
    # print(f"Text after cleaning:  {len(prep_text_before)}")
    prep_text_list = ' '.join(prep_text_before)
    # print(type(prep_text_list))


    return prep_text_list

# BOW

def get_top_n_words(corpus, n=None):
    corpus=[corpus]

    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    top_n_words_df = pd.DataFrame(words_freq, columns =['Word', 'Count'])

    return top_n_words_df.loc[:(n-1)]


def get_tf_idf(text, n=None):
    text = re.findall(r'(?:\d[.]|[^.])*(?:[.]|$)', text)
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(text)
    df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    return (df[:(n)])



[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# list_of_companies = os.listdir(
#     r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Sustainability Reports\To Do")

# for company in list_of_companies:
#     company_name = company
#     text = pdf_loader(company)
#     text = preprocessing_text(text)
#     result_df = topic_modeler(text)
#     result_df.to_csv(
#         fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Results\{company_name[:-4]}_FullText_TopicModeling.csv")

# for company in list_of_companies:
#     text_1, company_name, path, source = pdf_loader(company)
#     text_2 = preprocessing_text(text_1)
#     result_df = results_for_each_topic(text_2)
#     result_df.to_csv(
#         fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Results\{company_name}_FullText_Scoring.csv")

# frames = []

# for company in list_of_companies:
#     text_1, company_name, path, source = pdf_loader(company)
#     prep_text = preprocessing(text_1)
#     lemma_words = lemmatize_words(prep_text)
#     text_cleaned = remove_small_tokens(lemma_words)
#     top_n_words = get_top_n_words(text_cleaned, 20)
#     top_n_words = list(top_n_words.itertuples(index=False, name=None))
#     top_n_words
#     tf_idf = get_tf_idf(text_cleaned, 20)
#     tf_idf = tf_idf.reset_index()
#     tf_idf = tf_idf.rename(columns={"index": "Word"})
#     idf_list = list(tf_idf.itertuples(index=False, name=None))
#     idf_list
#     data = [(company_name, idf_list, top_n_words)]
#     df1 = pd.DataFrame(data, columns = ["Company Name", "TF-IDF", "Top N Words"])
#     frames.append(df1)
#     df_final = pd.concat(frames)
#     df_final.to_csv(fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Results\{company_name}_bow_tf_ifd.csv")

# for company in list_of_companies:
#     text_1, company_name, path, source = pdf_loader(company)
#     df_1 = preprocessing_df(text_1)
#     classification_ls, short_ls = classifier_pipeline(df_1)
#     nested_label_list = rebuilding(classification_ls, short_ls)
#     report_df = labeling_nested_list(nested_label_list)
#     results_df = scoring(report_df)
#     results_df.to_csv(
#         fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Results\{company_name}_Paragraphed_Cosine_Scoring.csv")
#     shutil.move(
#         fr"{path}", fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\Sustainability Reports\Already Done\{company_name}.pdf")



In [9]:
list_of_companies = os.listdir(r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Sustainability Reports\To Do")
list_of_companies

['2019-sustainability-report-doc-en.pdf',
 '2020-Annual-Report-7u42lsu22.pdf',
 '2020-responsibility-highlights-report.pdf',
 '2020-sustainability-report-doc-en.pdf',
 '20200625_man-es_pr_cr-report_2020_en.pdf',
 '2020_dnf_-_eng_0.pdf',
 '2020_Sustainability_Report.pdf',
 '2020_valora_geschaeftsbericht_de.pdf',
 '2021-03-22_JuliusBaer_CorporateSustainabilityReport2020_EN.pdf',
 '210928_UBP20Sustainability20Report.pdf',
 '2641133_DOWNLOAD.pdf',
 '5df03cc1-f2ec-65ed-9c3d-da3c343395a5.pdf',
 '5ZmOsI2P3oe0plCvOThrCySgcDcKCXqj.pdf',
 '668a8a26-d924-21aa-cd75-c12e5296ff2b.pdf',
 '9clin.pdf',
 'Allianz_Group_Sustainability_Report_2020-web.pdf',
 'Allreal_Sustainability_Report_EN_2020.pdf',
 'AMAG_Geschaeftsbericht_2021_Magazin_Englisch.pdf',
 'annual-and-sustainability-report-2020.pdf',
 'apollo-esg-report-v-12.pdf',
 'ar21e.pdf',
 'avaloq-csr-2020-report.pdf',
 'Bucher Sustainability report 2020.pdf',
 'c05179523.pdf',
 'cargill-aqua-nutrition-sustainability-report.pdf',
 'celgene-responsibi

In [10]:

list_of_companies = os.listdir(r"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Sustainability Reports\To Do")

for company in list_of_companies:
    ## BOW / TF-IDF
    try:
        print(f"We process now {company}")

        text_1, company_name, path, source = pdf_loader(company)
        prep_text = preprocessing(text_1)
        lemma_words = lemmatize_words(prep_text)
        text_cleaned = remove_small_tokens(lemma_words)
        top_n_words = get_top_n_words(text_cleaned, 20)
        top_n_words = list(top_n_words.itertuples(index=False, name=None))
        top_n_words
        tf_idf = get_tf_idf(text_cleaned, 20)
        tf_idf = tf_idf.reset_index()
        tf_idf = tf_idf.rename(columns={"index": "Word"})
        idf_list = list(tf_idf.itertuples(index=False, name=None))
        idf_list
        data = [(company_name, idf_list, top_n_words)]
        df1 = pd.DataFrame(data, columns = ["Company Name", "TF-IDF", "Top N Words"])
        # print(df1)
        company_name_new = f"{df1['Top N Words'][0][0][0]}_{df1['Top N Words'][0][1][0]}_{df1['Top N Words'][0][2][0]}"
        df1.to_csv(fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Results\Test\{company_name_new}_bow_tf_idf.csv")

        ## Full Text Topic Modeling

        # company_name = company
        # text = pdf_loader(company)
        # text = preprocessing_text(text)
        # result_df = topic_modeler(text)
        # result_df.to_csv(
        #     fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Results\Wiki Summary\{company_name_new}_FullText_TopicModeling.csv")

        ## Full Text Similarity Scoring

        text_1, company_name, path, source = pdf_loader(company)
        text_2 = preprocessing_text(text_1)
        result_df = results_for_each_topic(text_2)
        result_df.to_csv(
            fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Results\Test\{company_name_new}_FullText_Cosine_Scoring_Summary.csv")


        ## Paragraphed Similarity Scoring

        # text_1, company_name, path, source = pdf_loader(company)
        # df_1 = preprocessing_df(text_1)
        # classification_ls, short_ls = classifier_pipeline(df_1)
        # nested_label_list = rebuilding(classification_ls, short_ls)
        # report_df = labeling_nested_list(nested_label_list)
        # results_df = scoring(report_df)
        # results_df.to_csv(
        #     fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Results\Wiki Only\{company_name_new}_Paragraphed_Cosine_Scoring.csv")
        # shutil.move(
        #     fr"{path}", fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Sustainability Reports\Already Done\{company_name}.pdf")
        print(f"{company_name_new} SAFED")
    except:
        shutil.move(
            fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Sustainability Reports\To Do\{company}", fr"D:\Google Drive\Meine Ablage\HSLU\Master\Master Thesis\Coding Area Bulldozer\Data\Sustainability Reports\Not Working\{company}")
        print(f"{company} didn't work" )
        continue

We process now 2019-sustainability-report-doc-en.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


swiss_risk_sustainability SAFED
We process now 2020-Annual-Report-7u42lsu22.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


temenos_financial_year SAFED
We process now 2020-responsibility-highlights-report.pdf


...

chemical_mitsubishi_employee SAFED
We process now Allianz_Group_Sustainability_Report_2020-web.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Allianz_Group_Sustainability_Report_2020-web.pdf didn't work
We process now Allreal_Sustainability_Report_EN_2020.pdf


c:\Users\daveb\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


allreal_report_employee SAFED
We process now AMAG_Geschaeftsbericht_2021_Magazin_Englisch.pdf
